# climpy — Demo cu date reale

**Flux complet:** SST ERSSTv5 → EOF → AMO + Tripole → corelații cu SAT, SLP, Precipitații

Rulează celulele **în ordine**. Prima celulă instalează pachetele, a doua descarcă datele (~2 min).

In [ ]:
# ── Setup (rulează o singură dată per sesiune) ──────────────────────────────
import subprocess, sys

# Instalare dependențe
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                'cartopy', 'eofs', 'xarray', 'numpy', 'pandas',
                'matplotlib', 'scipy', 'netCDF4'], check=True)

# Adaugă climpy în path (evită problema cu setuptools)
import sys, os
repo_root = os.path.expanduser('~')   # pe Binder fișierele sunt în ~
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import climpy
print(f'climpy {climpy.__version__} — OK')

In [ ]:
# ── Descărcare date reale (~2 min pe Binder, ~1 min pe Colab) ──────────────
import os
os.makedirs('data/real', exist_ok=True)

datasets = {
    'data/real/sst.nc'   : 'https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc',
    'data/real/sat.nc'   : 'https://downloads.psl.noaa.gov/Datasets/gistemp/combined/250km/air.2x2.250.mon.anom.comb.nc',
    'data/real/precip.nc': 'https://downloads.psl.noaa.gov/Datasets/gpcc/full_v2020/precip.mon.total.1x1.v2020.nc',
    'data/real/slp.nc'   : 'https://downloads.psl.noaa.gov/Datasets/hadslp2/slp.mnmean.real.nc',
}

for dest, url in datasets.items():
    if os.path.exists(dest):
        print(f'  ✓ {dest} — already downloaded')
    else:
        print(f'  ↓ {dest} ...')
        os.system(f'wget -q --show-progress -O {dest} {url}')
        print(f'    done — {os.path.getsize(dest)//1024//1024} MB')

print('\nToate fișierele sunt gata.')

In [ ]:
# ── Imports ─────────────────────────────────────────────────────────────────
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import climpy

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

climpy.use_style('nature')
DATA = Path('data/real')
os.makedirs('figures', exist_ok=True)
print('OK')

## 1 — Preprocessing SST (ERSSTv5)

In [ ]:
sst_r = climpy.preprocess(
    DATA / 'sst.nc',
    var            = 'sst',
    lon_convention = '[-180,180]',
    fill_value     = -9.96921e+36,
    time           = ('1854-01', '2019-12'),
    lat            = (0, 80),
    lon            = (-76, 20),
    ref_period     = ('1951-01', '1980-12'),
    compute_annual = True,
    subtract_gm    = True,
)

annual_sst = sst_r['annual_no_gm'].dropna('year', how='all')
print('SST:', annual_sst.dims, dict(annual_sst.sizes))
print('Ani:', int(annual_sst.year[0]), '–', int(annual_sst.year[-1]))
print('NaN fraction:', float(annual_sst.isnull().mean().round(3)))

## 2 — Analiză EOF

In [ ]:
solver = climpy.EOF(annual_sst, min_variance=70)
solver.summary()

In [ ]:
eofs = solver.eofs()
pcs  = solver.pcs()
frac = solver.variance_fraction()

# AMO  = EOF1 + EOF2   (monopol uniform)
# Tripol = EOF1 - EOF2  (structură tri-polară)
eof_amo    = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=1, b= 1)
eof_tripol = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=1, b=-1)
pc_amo     = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=1, b= 1)
pc_tripol  = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=1, b=-1)

# Filtru 7 ani
pc_amo_ma7    = climpy.moving_average(pc_amo,    n=7, dim='year')
pc_tripol_ma7 = climpy.moving_average(pc_tripol, n=7, dim='year')

print('EOF-AMO shape:',    eof_amo.dims)
print('PC-AMO shape:',     pc_amo.dims)

## 3 — Figura EOF (AMO + Tripole)

In [ ]:
fig = climpy.ClimPlot(
    nrows=2, ncols=2,
    w=climpy.NATURE_2COL, h=5.5,
    map_proj=(climpy.Map(), climpy.Map(), 'ts', 'ts'),
)

fig[0].map(
    eof_amo,
    title=f'EOF-AMO ({float(frac[0]):.1f} %)',
    map_extent=[-76, 20, 0, 80],
    vmin=-0.5, vmax=0.5, cbar_step=0.2, nlevels=11,
    cbar_label='SST anomaly (°C)',
    show_land=True,
)
fig[1].map(
    eof_tripol,
    title=f'EOF-Tripole ({float(frac[1]):.1f} %)',
    map_extent=[-76, 20, 0, 80],
    vmin=-0.5, vmax=0.5, cbar_step=0.2, nlevels=11,
    cbar_label='SST anomaly (°C)',
    show_land=True,
)
fig[2].ts(
    pc_amo, pc_amo_ma7,
    title='PC-AMO', ylabel='Amplitude',
    labels=['annual', '7-yr MA'],
    colors=['#0077BB', '#CC3311'],
)
fig[3].ts(
    pc_tripol, pc_tripol_ma7,
    title='PC-Tripole', ylabel='Amplitude',
    labels=['annual', '7-yr MA'],
    colors=['#0077BB', '#CC3311'],
)
fig.label_subplots()
fig.savefig('figures/fig_eof_amo_tripole.pdf')
fig.show()

## 4 — Preprocessing SAT (GISTEMP)

In [ ]:
sat_r = climpy.preprocess(
    DATA / 'sat.nc',
    var            = 'air',
    lon_convention = '[-180,180]',
    time           = ('1880-01', '2015-12'),
    compute_annual = False,
    subtract_gm    = False,
)

# Medii sezoniere
sat_djfm = climpy.seasonal_means(sat_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
sat_jjas  = climpy.seasonal_means(sat_r['anom'], months=[6,7,8,9]).dropna('year', how='all')
print('SAT DJFM:', dict(sat_djfm.sizes))
print('SAT JJAS: ', dict(sat_jjas.sizes))

## 5 — Preprocessing Precipitații (GPCC)

In [ ]:
pr_r = climpy.preprocess(
    DATA / 'precip.nc',
    var            = 'precip',
    lon_convention = '[-180,180]',
    time           = ('1891-01', '2015-12'),
    compute_annual = False,
    subtract_gm    = False,
)

pr_djfm = climpy.seasonal_means(pr_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
pr_jjas  = climpy.seasonal_means(pr_r['anom'], months=[6,7,8,9]).dropna('year', how='all')
print('Precip DJFM:', dict(pr_djfm.sizes))
print('Precip JJAS: ', dict(pr_jjas.sizes))

## 6 — Preprocessing SLP (HadSLP2r)

In [ ]:
slp_r = climpy.preprocess(
    DATA / 'slp.nc',
    var            = 'slp',
    lon_convention = '[-180,180]',
    time           = ('1850-01', '2004-12'),
    compute_annual = False,
    subtract_gm    = False,
)

slp_djfm = climpy.seasonal_means(slp_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
print('SLP DJFM:', dict(slp_djfm.sizes))

## 7 — Corelații (cu intervalul comun automat)

In [ ]:
def align(pc, field):
    """Selectează intervalul comun de ani între PC și câmp."""
    common = sorted(set(pc['year'].values) & set(field['year'].values))
    return pc.sel(year=common), field.sel(year=common)

# AMO și Tripol × SAT DJFM
pc_a, sat_d  = align(pc_amo_ma7,    sat_djfm)
pc_t, sat_d2 = align(pc_tripol_ma7, sat_djfm)
r_amo_sat_djfm    = climpy.pearson_correlation(pc_a, sat_d,  dim='year')
r_tripol_sat_djfm = climpy.pearson_correlation(pc_t, sat_d2, dim='year')

# AMO și Tripol × SAT JJAS
pc_a, sat_j  = align(pc_amo_ma7,    sat_jjas)
pc_t, sat_j2 = align(pc_tripol_ma7, sat_jjas)
r_amo_sat_jjas    = climpy.pearson_correlation(pc_a, sat_j,  dim='year')
r_tripol_sat_jjas = climpy.pearson_correlation(pc_t, sat_j2, dim='year')

# AMO și Tripol × Precip DJFM
pc_a, pr_d  = align(pc_amo_ma7,    pr_djfm)
pc_t, pr_d2 = align(pc_tripol_ma7, pr_djfm)
r_amo_pr_djfm    = climpy.pearson_correlation(pc_a, pr_d,  dim='year')
r_tripol_pr_djfm = climpy.pearson_correlation(pc_t, pr_d2, dim='year')

# AMO și Tripol × SLP DJFM
pc_a, slp_d  = align(pc_amo_ma7,    slp_djfm)
pc_t, slp_d2 = align(pc_tripol_ma7, slp_djfm)
r_amo_slp_djfm    = climpy.pearson_correlation(pc_a, slp_d,  dim='year')
r_tripol_slp_djfm = climpy.pearson_correlation(pc_t, slp_d2, dim='year')

print('Corelații calculate.')

## 8 — Figura corelații SAT (DJFM + JJAS)

In [ ]:
CORR_KWARGS = dict(
    show_land=False,    # date vizibile pe uscat
    show_ocean=False,   # fond neutru
    vmin=-0.6, vmax=0.6,
    cbar_step=0.2,
    nlevels=13,
    cbar_label='r',
    cmap=plt.cm.RdBu_r,
)

fig = climpy.ClimPlot(
    nrows=2, ncols=2,
    w=climpy.NATURE_2COL, h=5.0,
    map_proj=(climpy.Map(-30), climpy.Map(-30),
              climpy.Map(-30), climpy.Map(-30)),
)

fig[0].map(r_amo_sat_djfm,    title='AMO × SAT — DJFM',    **CORR_KWARGS)
fig[1].map(r_tripol_sat_djfm, title='Tripole × SAT — DJFM', **CORR_KWARGS)
fig[2].map(r_amo_sat_jjas,    title='AMO × SAT — JJAS',    **CORR_KWARGS)
fig[3].map(r_tripol_sat_jjas, title='Tripole × SAT — JJAS', **CORR_KWARGS)

# Fond gri deschis pentru lizibilitate pe uscat
for ax in fig.axes:
    ax.set_facecolor('#F2F0EB')

fig.label_subplots()
fig.savefig('figures/fig_corr_sat.pdf')
fig.show()

## 9 — Figura corelații Precipitații DJFM

In [ ]:
fig = climpy.ClimPlot(
    nrows=1, ncols=2,
    w=climpy.NATURE_2COL, h=3.0,
    map_proj=(climpy.Map(-30), climpy.Map(-30)),
)

PRECIP_KWARGS = dict(
    show_land=False,
    show_ocean=False,
    vmin=-0.5, vmax=0.5,
    cbar_step=0.2,
    nlevels=11,
    cbar_label='r',
    cmap=plt.cm.BrBG,
)

fig[0].map(r_amo_pr_djfm,    title='AMO × Precip — DJFM',    **PRECIP_KWARGS)
fig[1].map(r_tripol_pr_djfm, title='Tripole × Precip — DJFM', **PRECIP_KWARGS)

for ax in fig.axes:
    ax.set_facecolor('#F2F0EB')

fig.label_subplots()
fig.savefig('figures/fig_corr_precip.pdf')
fig.show()

## 10 — Figura corelații SLP DJFM

In [ ]:
fig = climpy.ClimPlot(
    nrows=1, ncols=2,
    w=climpy.NATURE_2COL, h=3.0,
    map_proj=(climpy.Map(-30), climpy.Map(-30)),
)

SLP_KWARGS = dict(
    show_land=False,
    show_ocean=False,
    vmin=-0.5, vmax=0.5,
    cbar_step=0.2,
    nlevels=11,
    cbar_label='r',
    cmap=plt.cm.RdBu_r,
)

fig[0].map(r_amo_slp_djfm,    title='AMO × SLP — DJFM',    **SLP_KWARGS)
fig[1].map(r_tripol_slp_djfm, title='Tripole × SLP — DJFM', **SLP_KWARGS)

for ax in fig.axes:
    ax.set_facecolor('#F2F0EB')

fig.label_subplots()
fig.savefig('figures/fig_corr_slp.pdf')
fig.show()

---
## Figurile salvate
Toate figurile sunt în folderul `figures/` ca PDF vectorial.
Pe Binder: click pe fișier în panoul Files → Download.
Pe Colab: `from google.colab import files; files.download('figures/fig_eof_amo_tripole.pdf')`